# Questions to ask before you start:

1. Did you download the latest copy of the ADM Ops Log from Teams? Is the "Pleateau Name" column filled out correctly?
2. Do you have a file of the input poses ready? Does it contain the table of encoder info on the bottom right, and groundskeeping data in the top left?

## Did the script fail or give an error?
#### The most common cause of errors is if one of the input files is not formatted properly. 
1. The first thing I would check is the input pose spreadsheet. 
 * If that spreadsheet was created by a human, there's a chance it was formatted incorrectly and you'll have to do some copy/pasting into the "Poses - template.xlsx" file.
2. The second thing I would check is the ADM Ops Log. Specifically, the rows where the "Plateau" column matches the "[baseline/update]_ADM_plateau_name" variables below. 
 * This script pulls ONLY the rows that are indicated with a "Y" in the column titled "Final?". Did the operator mark the final measuremets approprately? 
 * Sometimes the ADM operator will perform repeated measurements for any number of reasons, so we make sure to mark the "best" one in this way. Make sure that for each plateau you have precisely 1 row with a "Y" for sMATF LED Pri, sMATF mirror, PATB LED Pri, and PATB mirror. Peter Morey can be of assistance in making sure the Ops Log is formatted correctly.

## 0 Parameter Definitions

#### 0.0.1 Parameters you'll need to tweak

In [1]:
baseline_ADM_plateau_name = 'Ti4' #Plateau name for the baseline ADM data.
update_ADM_plateau_name = 'WFI1'   #Plateau name for the updated ADM data
baseline_fname = 'files/Poses - Practice Input.xlsx' #What is the name of the input file? Don't forget to add "files/" if it's located in that subdirectory
#Name of the tab in the excel file that contains the data for your baseline poses. 
#If you're using a spreadsheet that was generated using this code, you'll want to set sheetname='update'. 
#If you're using a spreadsheet that came from Manal, chances are the tab is named 'input for Tyler analysis' or something
#sheetname = 'update'
#sheetname = 'Ti4C4 JV'
sheetname = 'Input for Tyler analysis'

#What do you want to name the output file?
#NOTE, the output spreadsheet from this code should not be used as an input for 'Pose Updater - FDPR.ipynb'.
output_filename = 'Poses - ADM Practice Output.xlsx'
#Remember the desired format is "Poses - [name of new circuit, i.e. BA2C3] ADM update from [name of baseline circuit, i.e. BA2C1] - [Your name].xlsx"

#### 0.0.2 Parameters you probably won't need to tweak

In [2]:
pat_target = 'B'         #Which PAT target do you want to use for the update? 
z_target = 'mirror'      #Do you want range data to come from the mirror or the retro? Options are "mirror" or "retro"
optimize_PAT_for_imperfect_nulling = True #Optimize the encoder positions for imperfect nulling between sMATF->PAT?

matf_led_type_for_baseline = 'pri'
matf_led_type_for_update = 'red'
pat_led_type = 'pri'
adm_log_filepath = 'files/ADM Ops Log.xlsx'

---
# Below here be math
---
### 0.1 Ingesting values from the template

In [3]:
%matplotlib notebook
import os
from adm_library import *

In [4]:
spreadsheet = pd.read_excel(baseline_fname,sheet_name=sheetname,skiprows=1,usecols='B:N')

focal_length = spreadsheet.iloc[0][0]-110 # focal length is track length -110mm, per TG
sMPA_to_WCS = spreadsheet.iloc[3,:3].astype(float)
sMPA_angle_to_WCS_deg = spreadsheet.iloc[6,:3].astype(float)
sMPA_angle_to_WCS_deg.name = 'sMPA XYZ Euler angle to WCS'
GSA_angle_WCS_deg = spreadsheet.iloc[7][0]
sMask=np.array(spreadsheet.iloc[10,1:4]).astype(float)

## Read in the poses themselves

In [5]:
spreadsheet = pd.read_excel(baseline_fname,sheet_name=sheetname,skiprows=15,usecols='B:S')

#Identify how many poses there are in the input spreadsheet, by identifying the first row at which there is no pose name
num_poses = np.where(pd.isnull(spreadsheet['Name.1']) == True)[0].min()

pose_actual = spreadsheet.iloc[0:num_poses,0:6]
pose_actual = pose_actual.set_index('Name')
pose_actual.columns = ['X','Y','Z','Rx','Ry']
X_5DOF,Y_5DOF,Z_5DOF,rx_deg_5DOF,ry_deg_5DOF = [[*pose_actual['X'].values.astype(float)],[*pose_actual['Y'].values.astype(float)],[*pose_actual['Z'].values.astype(float)],
                                                [*pose_actual['Rx'].values.astype(float)],[*pose_actual['Ry'].values.astype(float)]]

#Depending on the origin of the input spreadsheet, the desired encoder values are in a different place.
if sheetname == 'Input for Tyler analysis':
    pose_encoders = spreadsheet.iloc[0:num_poses,7:13]
elif sheetname == 'update':
    pose_encoders = spreadsheet.iloc[0:num_poses,12:18]
else:
    pose_encoders = spreadsheet.iloc[0:num_poses,7:13]
    
#Rename the columns because by default, each column header got a ".1" added to the end
new_columns=[]
for column in pose_encoders.columns:
    new_columns.append(column[:-2])
pose_encoders.columns = new_columns
pose_encoders = pose_encoders.set_index('Name')

print('Provided 5DOF position in real space: \n',pose_actual,'\n')
print('Provided 5DOF encoder values: \n',pose_encoders)


Provided 5DOF position in real space: 
                   X          Y          Z        Rx         Ry
Name                                                          
Pose 1     0.943594   1.280695  -0.338386  0.031453   0.035137
2 Pose  -109.505749 -45.867074  22.370475 -4.911732  10.680266
Pose 99  111.309055 -45.373337  22.058458 -4.857077 -10.600321
Pose 4   103.076801  59.765562  30.696044  5.307985 -10.311998
Pose 5   -101.43058   59.40225  30.809096  5.269783  10.410043
6 Pose     0.943594   1.280695  -0.338386  0.031453   0.035137
Pose 7  -109.505749 -45.867074  22.370475 -4.911732  10.680266
Pose 8   111.309055 -45.373337  22.058458 -4.857077 -10.600321
Pose 9   103.076801  59.765562  30.696044  5.307985 -10.311998
Pose 10  -101.43058   59.40225  30.809096  5.269783  10.410043
Pose 11    0.943594   1.280695  -0.338386  0.031453   0.035137
Pose 12 -109.505749 -45.867074  22.370475 -4.911732  10.680266
Pose 13  111.309055 -45.373337  22.058458 -4.857077 -10.600321
Pose 14  103.07

C:\Users\ebray\AppData\Local\Temp\ipykernel_16240\2840651906.py:1: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  spreadsheet = pd.read_excel(baseline_fname,sheet_name=sheetname,skiprows=15,usecols='B:S')


### 0.1.1 If both the pose_actual and pose_encoder arrays are provided in the spreadsheet above, convert between the two to make sure they both agree. 

In [6]:
#No point doing the following if one or the other array is completely empty
if (any(np.isfinite(pose_encoders.values.ravel().astype(float)))) and (any(np.isfinite(pose_actual.values.ravel().astype(float)))):

    print('(Calculated 5DOF position) - (Actual 5DOF position (from the spreadsheet)): ')
    calculated_pose_actual = pd.DataFrame(columns=pose_actual.columns)
    for index in pose_encoders.index:
        calculated_pose_actual = pd.concat((calculated_pose_actual,calculate_5DOF_from_encoders(pd.DataFrame(pose_encoders.loc[index]).T)))
    print(calculated_pose_actual-pose_actual,'\n\n')
    
    print('(Calculated 5DOF encoders) - (Actual 5DOF encoders (from the spreadsheet)): ')
    calculated_pose_encoders = pd.DataFrame(columns=pose_encoders.columns)
    for index in pose_encoders.index:
        calculated_pose_encoders = pd.concat((calculated_pose_encoders,calculate_encoders_from_5DOF(pd.DataFrame(pose_actual.loc[index]).T)))
    print(calculated_pose_encoders-pose_encoders)

(Calculated 5DOF position) - (Actual 5DOF position (from the spreadsheet)): 
                X         Y         Z        Rx        Ry
Pose 1  -0.001955  0.019764  0.051539  0.001420 -0.001320
2 Pose  -0.070318   0.17392  0.124407  0.007030  0.004108
Pose 99 -0.070835 -0.171766  0.015323 -0.007226  0.003743
Pose 4   0.071248  -0.15257 -0.001112 -0.008905 -0.005922
Pose 5   0.068846  0.195558  0.102315  0.009014 -0.004985
6 Pose  -0.001955  0.019764  0.051539  0.001420 -0.001320
Pose 7  -0.070318   0.17392  0.124407  0.007030  0.004108
Pose 8  -0.070835 -0.171766  0.015323 -0.007226  0.003743
Pose 9   0.071248  -0.15257 -0.001112 -0.008905 -0.005922
Pose 10  0.068846  0.195558  0.102315  0.009014 -0.004985
Pose 11 -0.001955  0.019764  0.051539  0.001420 -0.001320
Pose 12 -0.070318   0.17392  0.124407  0.007030  0.004108
Pose 13 -0.070835 -0.171766  0.015323 -0.007226  0.003743
Pose 14  0.071248  -0.15257 -0.001112 -0.008905 -0.005922
Pose 15  0.068846  0.195558  0.102315  0.009014 -0.00

### 0.2 If the "pose_actual" section of the supplied spreadsheet was empty, build a new "pose_actual" array based on the input "pose_encoder" array.

In [7]:
if not any(np.isfinite(pose_actual.values.astype(float)).ravel()):
    pose_actual = convert_pose_encoders_to_pose_actual(pose_encoders)
    print('NEWLY calculated 5DOF position in real space based on recorded 5DOF encoder values: \n',pose_actual)

# 1. Updating Poses with ADM Data

## 1.1 ADM Baseline  

#### 1.1.1 Input your baseline ADM values

In [8]:
p_null_PAT_baseline_encoder = get_data_from_ADM_log(baseline_ADM_plateau_name,z_type=z_target,pat_target = pat_target,
                                                    matf_led_type=matf_led_type_for_baseline,pat_led_type=pat_led_type,
                                                    index_name='PAT'+pat_target,print_details=True,filepath=adm_log_filepath) 
p_null_PAT_baseline = calculate_5DOF_from_encoders(p_null_PAT_baseline_encoder)

if pat_target=='B':
    p_null_offset = np.array([-90.,-110.,-10.]) #PATB is not on the POA boresight. It is offset by this much
elif pat_target=='A':
    p_null_offset = np.array([90.,-110.,-10.])
p_null_PAT_baseline_encoder


X                            88.57
Y                          108.385
Z                           -0.003
Rx                          0.1145
Ry                          0.0868
z_sMATF                    4023.52
z_PAT                      3474.01
z_type                      mirror
sMATF AC AZ               -0.00215
sMATF AC EL                 0.0022
PAT AC AZ                  -0.0022
PAT AC EL                   0.0032
sMATF LED X                 809.32
sMATF LED Y                600.175
PAT LED X                   809.33
PAT LED Y                    600.6
date           2023-08-12 00:00:00
Name: PATB, dtype: object


C:\Users\ebray\Miniconda3\envs\spyder-env\Lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,X,Y,Z,Rx,Ry,z_sMATF,z_PAT,z_type,sMATF AC AZ,sMATF AC EL,PAT AC AZ,PAT AC EL,sMATF LED X,sMATF LED Y,PAT LED X,PAT LED Y,date
PATB,88.57,108.385,-0.003,0.1145,0.0868,4023.52,3474.01,mirror,-0.00215,0.0022,-0.0022,0.0032,809.32,600.175,809.33,600.6,2023-08-12


### 1.1.2 Update the PAT pose to account for imperfect nulling.

In [9]:
#Apply a correction to account for imperfect nulling between PATB/sMATF and overwrite the values in the p_null_PAT_baseline_encoder DataFrame with the optimal ones.

#Save a copy of the p_null_PATB dataframe, which we'll write to an Excel file later
p_null_PAT_baseline_encoder_original = p_null_PAT_baseline_encoder.copy()
if optimize_PAT_for_imperfect_nulling == True:
    print(p_null_PAT_baseline_encoder.squeeze(),'\n')
    p_null_PAT_baseline_encoder = optimize_p_null_PAT_encoders(p_null_PAT_baseline_encoder,p_null_offset)
    p_null_PAT_baseline = calculate_5DOF_from_encoders(p_null_PAT_baseline_encoder)
    print('\n',p_null_PAT_baseline_encoder.squeeze())

X                            88.57
Y                          108.385
Z                           -0.003
Rx                          0.1145
Ry                          0.0868
z_sMATF                    4023.52
z_PAT                      3474.01
z_type                      mirror
sMATF AC AZ               -0.00215
sMATF AC EL                 0.0022
PAT AC AZ                  -0.0022
PAT AC EL                   0.0032
sMATF LED X                 809.32
sMATF LED Y                600.175
PAT LED X                   809.33
PAT LED Y                    600.6
date           2023-08-12 00:00:00
Name: PATB, dtype: object 

Starting GSARX/RY:  0.1145 0.0868
Optimal GSARX/RY:  0.1139747993 0.0859475535
Starting HTSA/VTSA:  88.57 108.385
Optimal HTSA/VTSA:  88.58216494999999 108.40170320000001

 X                        88.582165
Y                       108.401703
Z                           -0.003
Rx                        0.113975
Ry                        0.085948
z_sMATF                    40

#### 1.1.5 Form a dataframe with the values supplied so far

In [10]:
#Generate a dataframe where we will store some parameters for each pose
columns = ['X','Y','Z','Rx','Ry']
df = pose_actual
df['color']='crimson'

gsa_rot = R.from_euler('x',GSA_angle_WCS_deg, degrees=True)
df.loc['sMask',['X','Y','Z']] = gsa_rot.apply(sMask)
df.loc['sMask','color'] = 'g'

dz_baseline = p_null_PAT_baseline_encoder.loc['PAT'+pat_target,'z_sMATF'] - \
    p_null_PAT_baseline_encoder.loc['PAT'+pat_target,'z_PAT']
df = pd.concat([df,p_null_PAT_baseline])
print(p_null_PAT_baseline)
df.loc['PAT'+pat_target,'color']='darkblue'

df.loc['sMPA',['X','Y','Z']] = sMPA_to_WCS.values.astype(float)
df.loc['sMPA','color']='purple'

for pose in df.index:
    update_uvec(df,pose,length=focal_length,rotangle=GSA_angle_WCS_deg)
    if 'PDI' in pose:
        df.loc[pose,'color'] = 'yellow'

# create normal vector to sMPA
vec = np.array([0,0,1])
rotmat = R.from_euler('XYZ',sMPA_angle_to_WCS_deg, degrees=True)
df.loc['sMPA',['uvec_X','uvec_Y','uvec_Z']] = rotmat.apply(vec)

#ax = plot_poses(df)
#plot_sMPA(df, ax)
df

              X           Y         Z        Rx        Ry
PATB  88.906849  108.641166 -0.682675  0.113271  0.083863


,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
Pose 1,0.943594,1.280695,-0.338386,0.031453,0.035137,crimson,0.792405,248.143393,1268.071794
2 Pose,-109.505749,-45.867074,22.370475,-4.911732,10.680266,crimson,239.466788,350.312004,1220.458522
Pose 99,111.309055,-45.373337,22.058458,-4.857077,-10.600321,crimson,-237.694885,349.239187,1221.112189
Pose 4,103.076801,59.765562,30.696044,5.307985,-10.311998,crimson,-231.300660,128.369032,1264.754223
Pose 5,-101.43058,59.40225,30.809096,5.269783,10.410043,crimson,233.475701,129.171851,1264.272746
6 Pose,0.943594,1.280695,-0.338386,0.031453,0.035137,crimson,0.792405,248.143393,1268.071794
Pose 7,-109.505749,-45.867074,22.370475,-4.911732,10.680266,crimson,239.466788,350.312004,1220.458522
Pose 8,111.309055,-45.373337,22.058458,-4.857077,-10.600321,crimson,-237.694885,349.239187,1221.112189
Pose 9,103.076801,59.765562,30.696044,5.307985,-10.311998,crimson,-231.300660,128.369032,1264.754223
Pose 10,-101.43058,59.40225,30.809096,5.269783,10.410043,crimson,233.475701,129.171851,1264.272746


#### 1.1.6 Calculate the center of the sMATF flat in 5DOF coordinate system (currently assumes PATB is on-axis and aligned with GSA)

In [11]:
start_coords = np.array(df.loc['PAT'+pat_target,['X','Y','Z']])

rx = df.loc['PAT'+pat_target,'Rx']
ry = df.loc['PAT'+pat_target,'Ry']
print(rx,ry)
gsa_rot = R.from_euler('XY',[rx+GSA_angle_WCS_deg,ry], degrees=True)
uvec = np.array([0.,0.,dz_baseline]+p_null_offset)
rot_uvec = gsa_rot.apply(uvec) #np.dot(gsa_rot.as_matrix(),uvec)

sMATF_coords = start_coords+rot_uvec

df.loc['sMATF',['X','Y','Z']]=sMATF_coords
df.loc['sMATF',['Rx','Ry']]=df.loc['PAT'+pat_target,['Rx','Ry']]
df.loc['sMATF','color']='orange'
update_uvec(df,'sMATF',length=dz_baseline,rotangle=GSA_angle_WCS_deg)
#ax = plot_poses(df)
#plot_sMPA(df, ax)
df

0.11327069495913636 0.08386334137944938


,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
Pose 1,0.943594,1.280695,-0.338386,0.031453,0.035137,crimson,0.792405,248.143393,1268.071794
2 Pose,-109.505749,-45.867074,22.370475,-4.911732,10.680266,crimson,239.466788,350.312004,1220.458522
Pose 99,111.309055,-45.373337,22.058458,-4.857077,-10.600321,crimson,-237.694885,349.239187,1221.112189
Pose 4,103.076801,59.765562,30.696044,5.307985,-10.311998,crimson,-231.300660,128.369032,1264.754223
Pose 5,-101.43058,59.40225,30.809096,5.269783,10.410043,crimson,233.475701,129.171851,1264.272746
6 Pose,0.943594,1.280695,-0.338386,0.031453,0.035137,crimson,0.792405,248.143393,1268.071794
Pose 7,-109.505749,-45.867074,22.370475,-4.911732,10.680266,crimson,239.466788,350.312004,1220.458522
Pose 8,111.309055,-45.373337,22.058458,-4.857077,-10.600321,crimson,-237.694885,349.239187,1221.112189
Pose 9,103.076801,59.765562,30.696044,5.307985,-10.311998,crimson,-231.300660,128.369032,1264.754223
Pose 10,-101.43058,59.40225,30.809096,5.269783,10.410043,crimson,233.475701,129.171851,1264.272746


## 1.2 ADM Update

Now the system has undergone some kind of change that will cause the ADM measurements to be different, such as shimming the sWFI or cooling from ambient to operational temps. We need to use the new set of ADM sMATF/PATB measurements to update the calculated poses for PRs 1-5.

Now update all poses from this new PATB pose; this is the recipe that would be followed for real data

#### 1.2.1 Input new ADM data for the sMATF and PATB and convert to 5DOF actual space

In [12]:
p_null_PAT_update_encoder = get_data_from_ADM_log(update_ADM_plateau_name,z_type=z_target,pat_target = pat_target,
                                                  matf_led_type=matf_led_type_for_update,pat_led_type=pat_led_type,
                                                  index_name='PAT'+pat_target+'_update',print_details=True,filepath=adm_log_filepath)
#Identify whether we're working with the sMATF or MATF
if 'sMATF AC AZ' in p_null_PAT_update_encoder.columns:
    matf_target_update = 'sMATF'
elif 'MATF AC AZ' in p_null_PAT_update_encoder.columns:
    matf_target_update = 'MATF'

p_null_PAT_update = calculate_5DOF_from_encoders(p_null_PAT_update_encoder)
p_null_PAT_update

X                          87.188
Y                         107.072
Z                          -0.001
Rx                          0.069
Ry                         0.0613
z_MATF                  4021.0965
z_PAT                    3473.804
z_type                     mirror
MATF AC AZ               -0.00405
MATF AC EL               -0.00255
PAT AC AZ                 -0.0032
PAT AC EL                 -0.0024
MATF LED X                809.035
MATF LED Y                 599.27
PAT LED X                   809.2
PAT LED Y                   599.9
date          2023-09-14 00:00:00
Name: PATB_update, dtype: object


C:\Users\ebray\Miniconda3\envs\spyder-env\Lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,X,Y,Z,Rx,Ry
PATB_update,87.505724,107.311024,-0.668424,0.068283,0.059218


### 1.2.2 Update the PATB pose to account for imperfect nulling

In [13]:
#Apply a correction to account for imperfect nulling between PATB/sMATF and overwrite the values in the p_null_PAT_baseline_encoder DataFrame with the optimal ones.

#Save a copy of the p_null_PATB dataframe, which we'll write to an Excel file later
p_null_PAT_update_encoder_original = p_null_PAT_update_encoder.copy()

if optimize_PAT_for_imperfect_nulling == True:
    p_null_PAT_update_encoder = optimize_p_null_PAT_encoders(p_null_PAT_update_encoder,p_null_offset)
    p_null_PAT_update = calculate_5DOF_from_encoders(p_null_PAT_update_encoder)

Starting GSARX/RY:  0.069 0.0613
Optimal GSARX/RY:  0.06821453185000001 0.0616578296
Starting HTSA/VTSA:  87.188 107.072
Optimal HTSA/VTSA:  87.20024645 107.10111065000001


#### 1.2.2 Use the newly-provided data to calculate an updated position for PATB and sMATF

In [14]:
dz_update = p_null_PAT_update_encoder.loc['PAT'+pat_target+'_update','z_'+matf_target_update] - \
    p_null_PAT_update_encoder.loc['PAT'+pat_target+'_update','z_PAT'] + \
    (p_null_PAT_update_encoder.loc['PAT'+pat_target+'_update',['Y']]-p_null_PAT_baseline_encoder.loc['PAT'+pat_target,['Y']]).values[0]*np.sin(np.deg2rad(GSA_angle_WCS_deg+p_null_PAT_update_encoder.loc['PAT'+pat_target+'_update',['Rx']].astype(float).values[0]))
df = pd.concat([df,p_null_PAT_update])
df.loc['PAT'+pat_target+'_update','color'] = 'purple'
update_uvec(df,'PAT'+pat_target+'_update',length=focal_length,rotangle=GSA_angle_WCS_deg)

start_coords = np.array(df.loc['PAT'+pat_target+'_update',['X','Y','Z']])

rx = df.loc['PAT'+pat_target+'_update','Rx']
ry = df.loc['PAT'+pat_target+'_update','Ry']
gsa_rot = R.from_euler('XY',[rx+GSA_angle_WCS_deg,ry], degrees=True)
uvec = [0.,0.,dz_update]+p_null_offset # in GSA CS
rot_uvec = gsa_rot.apply(uvec) #np.dot(gsa_rot.as_matrix(),uvec)

sMATF_update_coords = start_coords+rot_uvec

df.loc['sMATF_update',['X','Y','Z']]=sMATF_update_coords
df.loc['sMATF_update',['Rx','Ry']]=df.loc['PAT'+pat_target+'_update',['Rx','Ry']]
df.loc['sMATF_update','color']='brown'
update_uvec(df,'sMATF_update',length=dz_update,rotangle=GSA_angle_WCS_deg)
#ax = plot_poses(df)
#plot_sMPA(df, ax)
print('Baseline poses')
df


Baseline poses


,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
Pose 1,0.943594,1.280695,-0.338386,0.031453,0.035137,crimson,0.792405,248.143393,1268.071794
2 Pose,-109.505749,-45.867074,22.370475,-4.911732,10.680266,crimson,239.466788,350.312004,1220.458522
Pose 99,111.309055,-45.373337,22.058458,-4.857077,-10.600321,crimson,-237.694885,349.239187,1221.112189
Pose 4,103.076801,59.765562,30.696044,5.307985,-10.311998,crimson,-231.300660,128.369032,1264.754223
Pose 5,-101.43058,59.40225,30.809096,5.269783,10.410043,crimson,233.475701,129.171851,1264.272746
6 Pose,0.943594,1.280695,-0.338386,0.031453,0.035137,crimson,0.792405,248.143393,1268.071794
Pose 7,-109.505749,-45.867074,22.370475,-4.911732,10.680266,crimson,239.466788,350.312004,1220.458522
Pose 8,111.309055,-45.373337,22.058458,-4.857077,-10.600321,crimson,-237.694885,349.239187,1221.112189
Pose 9,103.076801,59.765562,30.696044,5.307985,-10.311998,crimson,-231.300660,128.369032,1264.754223
Pose 10,-101.43058,59.40225,30.809096,5.269783,10.410043,crimson,233.475701,129.171851,1264.272746


In [15]:
#How did the distance between PATB and sMATF change from pre- to post-environment shift?
print('PAT'+pat_target+'->sMATF distance changed by: ',dz_update-dz_baseline, ' mm')

PATB->sMATF distance changed by:  -1.966148263583591  mm


## 1.3 Calculate sMATF and PAT deltas

In [16]:
sMATF_delta = df.loc['sMATF_update',['X','Y','Z']] - df.loc['sMATF',['X','Y','Z']]
#sMATF_delta.loc['Z'] += 
print('sMATF (X,Y,Z) deltas: \n',sMATF_delta,'\n')

patb_delta = df.loc['PAT'+pat_target+'_update',['X','Y','Z','Rx','Ry']] - df.loc['PAT'+pat_target,['X','Y','Z','Rx','Ry']]
#patb_drxry = df.loc['PATB_update',['Rx','Ry']] - df.loc['PATB',['Rx','Ry']]
print('PAT'+pat_target+' (X,Y,Z,Rx,Ry) deltas: \n',patb_delta,'\n')

sMATF (X,Y,Z) deltas: 
 X   -1.619569
Y   -1.244652
Z   -1.948988
dtype: object 

PATB (X,Y,Z,Rx,Ry) deltas: 
 X    -1.388777
Y    -1.300994
Z     0.014062
Rx   -0.045773
Ry   -0.024288
dtype: object 



## 1.4 Transform old poses to the new sMATF reference frame

### 1.4.1 Apply a translation to all poses

In [17]:
df_update = df.copy()
T = df.loc['sMATF_update',['X','Y','Z']]
df_update[['X','Y','Z']] -= T
print('Translation required to put sMATF_update at the origin: \n',T)

Translation required to put sMATF_update at the origin: 
 X     -1.922949
Y    102.291933
Z    548.082566
Name: sMATF_update, dtype: object


### 1.4.2 Now perform 3 rotations to all the poses:
1. To align to the sMATF frame
2. To rotate everything by delta(Rx,Ry)
3. To undo #1

In [18]:
update_rot1 = R.from_euler('XY',[df.loc['PAT'+pat_target,'Rx']+GSA_angle_WCS_deg,df.loc['PAT'+pat_target,'Ry']], degrees=True)
update_rot2 = R.from_euler('XY',[patb_delta.loc['Rx'],patb_delta.loc['Ry']], degrees=True)
update_rot_combined = update_rot1 * update_rot2 * update_rot1.inv()
                                
for pose in df_update.index:
                                
    df_update.loc[pose,['X','Y','Z']]=update_rot_combined.apply(df_update.loc[pose,['X','Y','Z']].astype(float)) #np.dot(update_rot.as_matrix(),df_update.loc[pose,['X','Y','Z']])
    df_update.loc[pose,['uvec_X','uvec_Y','uvec_Z']] = update_rot_combined.apply(df_update.loc[pose,['uvec_X','uvec_Y','uvec_Z']].astype(float)) 
    if pose not in ['sMask','sMPA']:
        update_RxRy(df_update,pose,GSA_angle_WCS_deg)


### 1.4.3 Translate back to the 5DOF frame by undoing the translation from Cell 1.4.1, followed by adding the sMATF delta(X,Y,Z) offset from Cell 1.3

In [19]:
df_update[['X','Y','Z']] += T
df_update.loc[df_update.index[:12],['X','Y','Z']] += update_rot_combined.apply(sMATF_delta.astype(float)) #np.dot(update_rot.as_matrix(),delta)
#ax = plot_poses(df_update)
#plot_sMPA(df_update, ax)
print('Updated poses')
df_update

Updated poses


,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
Pose 1,-0.438692,-0.403497,-2.204939,-0.014320,0.010849,crimson,0.244667,249.156405,1267.873366
2 Pose,-110.893606,-47.542198,20.495651,-4.957116,10.656065,crimson,238.930458,351.306587,1220.277741
Pose 99,109.921268,-47.030539,20.275062,-4.903232,-10.624523,crimson,-238.231356,350.195025,1220.733842
Pose 4,101.676804,58.114549,28.825225,5.262625,-10.336180,crimson,-231.837180,129.360332,1264.554971
Pose 5,-102.830575,57.734498,28.853525,5.223597,10.385859,crimson,232.939274,130.201015,1264.266126
6 Pose,-0.438692,-0.403497,-2.204939,-0.014320,0.010849,crimson,0.244667,249.156405,1267.873366
Pose 7,-110.893606,-47.542198,20.495651,-4.957116,10.656065,crimson,238.930458,351.306587,1220.277741
Pose 8,109.921268,-47.030539,20.275062,-4.903232,-10.624523,crimson,-238.231356,350.195025,1220.733842
Pose 9,101.676804,58.114549,28.825225,5.262625,-10.336180,crimson,-231.837180,129.360332,1264.554971
Pose 10,-102.830575,57.734498,28.853525,5.223597,10.385859,crimson,232.939274,130.201015,1264.266126


# 1.4 >>>>Results<<<<

#### 1.4.1 Converting poses from real space to encoder space

In [20]:
ignored_poses = ['sMask','PATB','sMPA','sMATF','PATB_update','sMATF_update']
df_update_encoder_space = convert_df_to_encoder_space(df_update,pose_select='',ignored_poses=ignored_poses)
print('Poses calculated using PAT'+pat_target+' and the PAT '+z_target)
print('Updated Poses (in encoder space): \n',df_update_encoder_space,'\n\n')

print('Baseline Poses (in encoder space), just for easy reference: \n',pose_encoders)

Poses calculated using PATB and the PAT mirror
Updated Poses (in encoder space): 
                   X          Y          Z        Rx         Ry
Pose 1    -0.462639  -0.420593  -2.181261 -0.013581   0.012929
2 Pose  -110.777325 -47.750127  20.478623 -4.957125  10.655066
Pose 99  109.877611 -46.788664  20.408543 -4.901766 -10.624286
Pose 4   101.483456  58.317422  29.035361  5.267480 -10.333757
Pose 5  -102.872948  57.495288  28.920212  5.222995  10.386689
6 Pose    -0.462639  -0.420593  -2.181261 -0.013581   0.012929
Pose 7  -110.777325 -47.750127  20.478623 -4.957125  10.655066
Pose 8   109.877611 -46.788664  20.408543 -4.901766 -10.624286
Pose 9   101.483456  58.317422  29.035361  5.267480 -10.333757
Pose 10 -102.872948  57.495288  28.920212  5.222995  10.386689
Pose 11   -0.462639  -0.420593  -2.181261 -0.013581   0.012929
Pose 12 -110.777325 -47.750127  20.478623 -4.957125  10.655066
Pose 13  111.490903 -45.500414  22.334025 -4.901766 -10.624286
Pose 14  103.100577  59.572310  30.

#### (OPTIONAL) 1.4.2 Display some other tables that may be helpful for diagnostics

In [21]:
print_extra_diagnostics = True
poses_to_display = [val for val in df.index if val not in ignored_poses]+['sMask', 'sMPA']

if print_extra_diagnostics == True:
    print('Baseline Poses (in real space): \n',df.loc[poses_to_display,['X','Y','Z','Rx','Ry']],'\n\n')
    print('Updated Poses (in real space): \n',df_update.loc[poses_to_display,['X','Y','Z','Rx','Ry']], '\n\n')
    print('Deltas between Baseline/Updated poses (in real space): \n',-df.loc[poses_to_display,['X','Y','Z','Rx','Ry']] + df_update.loc[poses_to_display,['X','Y','Z','Rx','Ry']], '\n\n')
    print('Change in calculated endpoint positions (5DOF frame): \n',compute_endpoint_errors(df,df_update,pose_select='',ignored_poses=ignored_poses),'\n\n')
    if sheetname == 'Input for Tyler analysis':
        print('Change in calculated endpoint positions (sMPA frame): \n',compute_endpoint_errors_sMPA_frame(df,df_update,pose_select='',ignored_poses=ignored_poses),'\n\n')
        print('Distance from pose focus to sMPA: \n\n',compute_distance_to_sMPA(df,pose_select=''),'\n\n')  
    print('Verify that all poses intersect at the pupil in the same way before and after the update:')
    print(check_pupil_crossing([df,df_update]),'\n\n')
    

Baseline Poses (in real space): 
                   X           Y          Z        Rx         Ry
Pose 1     0.943594    1.280695  -0.338386  0.031453   0.035137
2 Pose  -109.505749  -45.867074  22.370475 -4.911732  10.680266
Pose 99  111.309055  -45.373337  22.058458 -4.857077 -10.600321
Pose 4   103.076801   59.765562  30.696044  5.307985 -10.311998
Pose 5   -101.43058    59.40225  30.809096  5.269783  10.410043
6 Pose     0.943594    1.280695  -0.338386  0.031453   0.035137
Pose 7  -109.505749  -45.867074  22.370475 -4.911732  10.680266
Pose 8   111.309055  -45.373337  22.058458 -4.857077 -10.600321
Pose 9   103.076801   59.765562  30.696044  5.307985 -10.311998
Pose 10  -101.43058    59.40225  30.809096  5.269783  10.410043
Pose 11    0.943594    1.280695  -0.338386  0.031453   0.035137
Pose 12 -109.505749  -45.867074  22.370475 -4.911732  10.680266
Pose 13  111.309055  -45.373337  22.058458 -4.857077 -10.600321
Pose 14  103.076801   59.765562  30.696044  5.307985 -10.311998
Pose 1

# 2 Write the new poses to an Excel file in the current directory

In [22]:
write_to_excel = True

if write_to_excel == True:
    write_new_poses_to_Excel(output_filename,'',update_type='ADM',baseline_filepath=baseline_fname,
                             columns=columns,sMPA_angle_to_WCS_deg=sMPA_angle_to_WCS_deg,GSA_angle_WCS_deg=GSA_angle_WCS_deg,
                             baseline_ADM_plateau_name=baseline_ADM_plateau_name,update_ADM_plateau_name=update_ADM_plateau_name,
                             df=df,df_encoders=pose_encoders,df_update=df_update,df_update_encoders=df_update_encoder_space,
                            focal_length=focal_length,num_poses=num_poses,p_null_PAT_baseline_encoder_original=p_null_PAT_baseline_encoder_original,
                            p_null_PAT_update_encoder_original=p_null_PAT_update_encoder_original)

**Writing to Excel complete.**
**Filename:  Poses - ADM Practice Output.xlsx
